## Main RBC Trad MPF Workflow

### Import Packages and Functions
* Imports relevant Python packages.
* Imports relevant Python functional scripts. 

In [1]:
# Import Python Packages
import pandas as pd
import numpy as np
from pandas_schema import Column, Schema 
from pandas_schema.validation import LeadingWhitespaceValidation, TrailingWhitespaceValidation, CanConvertValidation, MatchesPatternValidation, InRangeValidation, InListValidation, IsDtypeValidation

In [2]:
# Import supporting functional methods
import mpf_trad_functions_phase1 as mpf_trad_func_ph1
import mpf_trad_functions_phase2 as mpf_trad_func_ph2
import mpf_trad_functions_phase3 as mpf_trad_func_ph3
import output_mpf as output_mpf
import mpf_trad_schema as mpf_trad_sch
from common_data_reconciliation import Data_Reconciliation

### Import Constant Variables
* This step defines constant parameters and expected input schema of the valuation data (for validation in Stage 1a below).
* We expect some of these variables would be better suited to be defined as a configuration input in the FDR (e.g. valuation date), rather than a current input in this notebook.

In [3]:
# Define Constant Parameters
val_date_const = pd.to_datetime('31/07/2024', dayfirst=True)

In [4]:
# Define Input Data Schema
# For all columns, check for leading and trailing white space
input_schema = mpf_trad_sch.input_trad_schema()

In [5]:
# Define Output Data Schema
output_schema = mpf_trad_sch.prophet_output_trad_schema()

### Import Data
* Currently this step imports "csv" file data from a local drive repository.
* We expect this to be relinked to the Data team's FDR landing spot source. 

In [6]:
# Import MPF Input Extracts and other input files
usecol_list = mpf_trad_sch.input_cols_to_keep()
poldata_csv = pd.read_csv('Inputs/TRAD_extracts.csv', usecols = usecol_list, dtype = {'BENPLN': 'string', 'AGNTNUM': 'string'})

# Read Contract Type input switch table and Product Type output table
# LOB = 'TRAD'
# CNTTYPE_input_switch = pd.read_csv('Inputs/INPUT_SWITCH.csv')
# prod_output_switch = pd.read_csv('Inputs/OUTPUT_SWITCH.csv')

# Import Grouping table
grouping_table_df = pd.read_csv('Inputs/Grouping.csv', header=0)

# Import Reference tables
dmtm_cellpc_ind_tbl = pd.read_csv('Inputs/DMTM_CELLPC_IND.csv', header=0)
dmtm_tbl = pd.read_csv('Inputs/DMTM_TABLE.csv', header=0)
general_tbl = pd.read_csv('Inputs/General.csv', header=0, keep_default_na = False)
edp_polno_ind_tbl = pd.read_csv('Inputs/EDP_POLNO_IND.csv', header=0)
rb_tbl = pd.read_csv('Inputs/RB.csv', header=0)
trad_wv_type_tbl = pd.read_csv('Inputs/TRAD_WV_TYPE.csv', header=0)
ri_ced_trad_tbl = pd.read_csv('Inputs/RI_CED_TRAD.csv', header=0)
lookup_list_tbl = pd.read_csv('Inputs/LOOKUP_LIST.csv', header=0)
benefit_code_tbl = pd.read_csv('Inputs/IFRS17_BENEFIT_CODE.csv', header = 0)
clib_list_tbl = pd.read_csv('Inputs/Output_CLib.csv', header = 0)
par_channel_tbl = pd.read_csv('Inputs/PAR_CHANNEL.csv', header = 0)
par_table_category_tbl = pd.read_csv('Inputs/PAR_Table_Category.csv', header = 0)
par_table_tbl = pd.read_csv('Inputs/PAR_TABLE.csv', header = 0)
asrf_tbl = pd.read_csv('Inputs/ASRF_MPF_TABLE.csv', header = 0, dtype = {'Polno': "string"})

# Instantiate the class for data reconciliation
data_rec = Data_Reconciliation()

### Stage 1a: Validating Data
* Performs initial check of the imported data's schema against an expected schema / format and fields.
* We expect the schemas to be aligned with the Data team's FDR landing spot schema. 

In [7]:
# Apply validation of expected schema (column names, and data types)
input_validation_error_list = input_schema.validate(poldata_csv)

# Print results of validation errors
if input_validation_error_list == []:
    print("No data validation errors")
else: 
    for error in input_validation_error_list:
        print(error)

# Set up validation table and valuation extract
data_rec.add_row_agg(poldata_csv)

The column HISSDTE has a dtype of object which is not a subclass of the required type <class 'numpy.datetime64'>


### Stage 1b: Filter Contract Type

In [8]:
# Apply filter on poldata for selected CNTTYPE in INPUT_SWITCH table
# poldata_csv = comm_func.input_filter_contract_type(CNTTYPE_input_switch, poldata_csv, LOB)

### Stage 1c: Data Standardisation
* Performs standardisation of data fields (conversion to data types where required). 
* We expect these standrdisations to be performed before this process (i.e. in the Data team's FDR landing spot).

In [9]:
# Assign raw csv dataframe to the poldata_df variable (which will persist for all subsequent transformations)
poldata_df = poldata_csv

# Preprocessing step to turn columns to datetime or apply explicit data formatting
poldata_df['HISSDTE'] = pd.to_datetime(poldata_df['HISSDTE'])
poldata_df['PROPRCDT'] = pd.to_datetime(poldata_df['PROPRCDT'])
poldata_df['ZNPDD'] = pd.to_datetime(poldata_df['ZNPDD'])
poldata_df['AGNTNUM'] = poldata_df['AGNTNUM'].astype(str)

### Stage 2: Enrich Data

#### Phase 1: Preparation for grouped level calculations
* Prepare the valuation data for grouped level calculations by assigning new and updating existing variables to enable mapping of the product grouping table.
* Corresponds to Alteryx containers 3-4.

In [10]:
# Record dataframe metrics
data_rec.add_row_agg(poldata_df)

# Alteryx Container 3 (Steps 7.8-7.25)
poldata_df = mpf_trad_func_ph1.add_col_record_to_keep(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_file(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_pol_number(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_extmonth(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_dur1(poldata_df, val_date_const)
poldata_df = mpf_trad_func_ph1.add_col_dur2(poldata_df, val_date_const)
poldata_df = mpf_trad_func_ph1.add_col_durationif_m(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_entry_year(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_entry_month(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_issue_month(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_issue_year(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_occ_class(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_ben_plan(poldata_df) 
poldata_df = mpf_trad_func_ph1.upd_col_benpln(poldata_df) 
poldata_df = mpf_trad_func_ph1.add_col_banca(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_agency(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_annual_prem(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_single_prem(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_mpf_ind(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_basic_psm_ind(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_polyear(poldata_df, val_date_const)
poldata_df = mpf_trad_func_ph1.add_col_recordmpf(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_enhanced_ind(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_tot_ap(poldata_df)

# Add to Data Reconciliation table (Phase 1 Method 1)
data_rec.add_row_agg(poldata_df)

# Store TOTAP_TAB E2E table for future output
totap_df = mpf_trad_func_ph1.output_totap_tab(poldata_df)

poldata_df = mpf_trad_func_ph1.add_col_totap_sumins(poldata_df)
poldata_df = mpf_trad_func_ph1.upd_col_enhanced_ind_dmtm(poldata_df)
poldata_df = mpf_trad_func_ph1.upd_col_banca_dmtm(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_sb_s_yr(poldata_df)
poldata_df = mpf_trad_func_ph1.mrg_dmtm_cellpc_ind_tbl(poldata_df, dmtm_cellpc_ind_tbl)
poldata_df = mpf_trad_func_ph1.mrg_dmtm_tbl(poldata_df, dmtm_tbl)
poldata_df = mpf_trad_func_ph1.add_col_min_rid(poldata_df)
poldata_df = mpf_trad_func_ph1.upd_col_sum_assured_min_rid(poldata_df)
poldata_df = mpf_trad_func_ph1.upd_col_acc_sum_assured(poldata_df)
poldata_df = mpf_trad_func_ph1.upd_col_incr_ben(poldata_df)
poldata_df = mpf_trad_func_ph1.upd_col_yi_dth_tpd(poldata_df)
poldata_df = mpf_trad_func_ph1.upd_col_sum_assured_nondmtm(poldata_df, general_tbl)
poldata_df = mpf_trad_func_ph1.upd_col_lformdesc(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_sumins_freq(poldata_df)
poldata_df = mpf_trad_func_ph1.mrg_edp_polno_ind_tbl(poldata_df, edp_polno_ind_tbl)
poldata_df = mpf_trad_func_ph1.add_col_fundtype(poldata_df)
poldata_df = mpf_trad_func_ph1.mrg_rb_tbl(poldata_df, rb_tbl)
poldata_df = mpf_trad_func_ph1.add_col_old_new_series_product_ind(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_old_series_pre_edp_fundtype(poldata_df)
poldata_df = mpf_trad_func_ph1.mrg_trad_wv_type_tbl(poldata_df, trad_wv_type_tbl) 
poldata_df = mpf_trad_func_ph1.mrg_ri_ced_trad_tbl(poldata_df, ri_ced_trad_tbl)

# # Alteryx Container 4 (Steps 7.26-7.35)
poldata_df = mpf_trad_func_ph1.add_col_pm_ind(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_filtered_cc_ann_prem(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_ccb_cc_ann_prem(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_ccb_cc_sum_assd(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_ccb_cc_term_y(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_mrta_fields(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_polfee_annprem(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_mme_prm(poldata_df)

# Store BLANKSA_TAB, PRUTERM_TAB_STAGE0 and PRUTERM_TAB E2E tables for future output
poldata_df, blanksa_tab_df = mpf_trad_func_ph1.add_col_temp_sa(poldata_df)
pruterm_tab_df_stage0 = mpf_trad_func_ph1.output_pruterm_tab_stage0_table(poldata_df)
poldata_df, pruterm_tab_df = mpf_trad_func_ph1.add_col_pruterm_basic(poldata_df)

poldata_df = mpf_trad_func_ph1.upd_col_enhanced_ind(poldata_df)
poldata_df = mpf_trad_func_ph1.add_col_prod_name(poldata_df, dmtm_tbl, general_tbl)

# Add to Data Reconciliation table (Phase 1 Method 2)
data_rec.add_row_agg(poldata_df)

# Phase 1 Testing Output
poldata_df.to_csv('./Outputs/Python_Phase1_output.csv')

2024-10-18 10:42:08: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_record_to_keep' END
2024-10-18 10:42:09: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_file' END
2024-10-18 10:42:09: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_pol_number' END
2024-10-18 10:42:09: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_extmonth' END
2024-10-18 10:42:09: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_dur1' END
2024-10-18 10:42:09: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_dur2' END
2024-10-18 10:42:09: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_durationif_m' END
2024-10-18 10:42:09: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_entry_year' END
2024-10-18 10:42:09: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_entry_month' END
2024-10-18 10:42:09: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_issue_month' END
2024-10-18 1

#### Phase 2: Grouped level calculations
* Perform product level (grouped) calculations on the valuation extract records. 
* Corresponds to Alteryx containers 5-6 (i.e. the existing FGR calculation methods).

In [11]:
# Stage 2: Grouped calculations

# Add to Data Reconciliation table (Start of Phase 2)
data_rec.add_row_agg(poldata_df)

# Add all variables where G0 = 0
poldata_df = mpf_trad_func_ph2.add_col_group_zero(poldata_df)

# Rank 1
poldata_df = mpf_trad_func_ph2.add_col_age2_atentry(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_annual_prem_1(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_as_curr_mth_1(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_as_curr_year_1(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_basic(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_blanksa(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_cc_ann_prem(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_comm_ind(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_comp_ben_sa(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_defer_per_y(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_gstprem(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_incsa_pct(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_init_pols_if(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_mme_prem(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_mom_i(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_mort_int_pc(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_no_ls_claim(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_nonpar_prem(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_pol_term_y(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_ppt_code(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_prem_freq(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_prem_months(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_prem_paybl_y_1(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_retirement_age(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_sex(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_sex2(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_smoker_stat(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_smoker2_stat(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_substd_prem(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_treaty_id_treaty(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_age_at_entry_1(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.upd_col_fundtype(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_prem_mode(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_series_ind(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.upd_col_single_prem(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_cc_term_y(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.upd_col_zrevbns(poldata_df, grouping_table_df, val_date_const)

# Add to Data Reconciliation table (Phase 2 Rank 1 Before Adjustment)
data_rec.add_row_agg(poldata_df)

# Adjustment after Rank 1
poldata_df = mpf_trad_func_ph2.add_col_dth_mult(poldata_df, lookup_list_tbl)
poldata_df = mpf_trad_func_ph2.upd_col_ppt_code(poldata_df)

# Phase 2 Rank 1 Testing Output
poldata_df.to_csv('./Outputs/Python_Phase2_Rank1_output.csv')

# Add to Data Reconciliation table (Phase 2 Rank 1 After Adjustment)
data_rec.add_row_agg(poldata_df)

# Rank 2
poldata_df = mpf_trad_func_ph2.add_col_death_ben(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_init_decb_if(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_link_prem(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_waived_ind(poldata_df, grouping_table_df) 
poldata_df = mpf_trad_func_ph2.add_col_waived_prem(poldata_df, grouping_table_df) 
poldata_df = mpf_trad_func_ph2.add_col_age_at_entry_2(poldata_df, grouping_table_df) 
poldata_df = mpf_trad_func_ph2.add_col_prem_paybl_y_2(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_mat_ben_pp_1(poldata_df, grouping_table_df) 
poldata_df = mpf_trad_func_ph2.add_col_plan_code(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_basic_ind(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.upd_col_cc_sum_assd(poldata_df, grouping_table_df) 
poldata_df = mpf_trad_func_ph2.upd_col_sum_assured_grouped(poldata_df, grouping_table_df) 
# Phase 2 Rank 2 Testing Output
poldata_df.to_csv('./Outputs/Python_Phase2_Rank2_output.csv')

# Add to Data Reconciliation table (Phase 2 Rank 2)
data_rec.add_row_agg(poldata_df)

# Rank 3
poldata_df = mpf_trad_func_ph2.add_col_profile(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_waived_term(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.upd_col_product_name(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.upd_col_cic_ben_pp(poldata_df, grouping_table_df)
# Phase 2 Rank 3 Testing Output
poldata_df.to_csv('./Outputs/Python_Phase2_Rank3_output.csv')

# Add to Data Reconciliation table (Phase 2 Rank 3)
data_rec.add_row_agg(poldata_df)

# Rank 4
poldata_df = mpf_trad_func_ph2.add_col_as_curr_mth_2(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_as_curr_year_2(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_annual_prem_2(poldata_df, grouping_table_df)
# Phase 2 Rank 4 Testing Output
poldata_df.to_csv('./Outputs/Python_Phase2_Rank4_output.csv')

# Add to Data Reconciliation table (Phase 2 Rank 4)
data_rec.add_row_agg(poldata_df)

# Rank 5
poldata_df = mpf_trad_func_ph2.add_col_g_mat_ben_pp(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_rider_ben_pp(poldata_df, grouping_table_df)
poldata_df = mpf_trad_func_ph2.add_col_mat_ben_pp_2(poldata_df, grouping_table_df)
# Phase 2 Rank 5 Testing Output
poldata_df.to_csv('./Outputs/Python_Phase2_Rank5_output.csv')

# Add to Data Reconciliation table (Phase 2 Rank 5)
data_rec.add_row_agg(poldata_df)

# Store BASIC_POLFEE E2E table for future output
basic_polfee_df = mpf_trad_func_ph2.output_basic_polfee(poldata_df)

# Store BLANKSA E2E tables for future output
filtered_elsp_df, filtered_elrp_df, filtered_mrta_rop_df, filtered_mrta_df = mpf_trad_func_ph2.output_blanksa_summary(poldata_df)

2024-10-18 10:42:10: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_group_zero' END
2024-10-18 10:42:10: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_age2_atentry' END
2024-10-18 10:42:10: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_annual_prem_1' END
2024-10-18 10:42:10: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_as_curr_mth_1' END
2024-10-18 10:42:10: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_as_curr_year_1' END
2024-10-18 10:42:10: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_basic' END
2024-10-18 10:42:10: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_blanksa' END
2024-10-18 10:42:10: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_cc_ann_prem' END
2024-10-18 10:42:10: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_comm_ind' END
2024-10-18 10:42:10: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_comp_ben_sa' 

#### Phase 3: Preparation for outputting MPFs
* Perform post grouped calculation enrichments, to prepare the data for MPF generation.
* Corresponds to Alteryx containers 7, 8 and 10.

In [12]:
# Add to Data Reconciliation table (End of Phase 2 / Start of Phase 3)
data_rec.add_row_agg(poldata_df)

# Preparation of outputs following grouped level calculations
poldata_df = mpf_trad_func_ph3.add_col_age_at_entry(poldata_df)
poldata_df = mpf_trad_func_ph3.add_col_as_curr_mth(poldata_df)
poldata_df = mpf_trad_func_ph3.add_col_as_curr_year(poldata_df)
poldata_df = mpf_trad_func_ph3.add_col_mat_ben_pp(poldata_df)
poldata_df = mpf_trad_func_ph3.add_col_prem_payabl_y(poldata_df)
poldata_df = mpf_trad_func_ph3.add_col_bft_paybl_y(poldata_df)
poldata_df = mpf_trad_func_ph3.upd_col_annual_prem(poldata_df)
poldata_df = mpf_trad_func_ph3.upd_col_sum_assured_prod(poldata_df)
poldata_df = mpf_trad_func_ph3.add_col_polt(poldata_df)
poldata_df = mpf_trad_func_ph3.add_col_cfstat(poldata_df)
poldata_df = mpf_trad_func_ph3.add_col_spcode(poldata_df)
poldata_df = mpf_trad_func_ph3.upd_col_spcode(poldata_df, val_date_const)

# Add to Data Reconciliation table (before MPF transformation)
data_rec.add_row_agg(poldata_df)

# Store CA_SUMMARY E2E table for future output
ca_summary_df = mpf_trad_func_ph3.output_ca_summary(poldata_df)

poldata_df = mpf_trad_func_ph3.upd_col_for_ca(poldata_df)
poldata_df = mpf_trad_func_ph3.add_col_wlist_total_prem(poldata_df)
poldata_df = mpf_trad_func_ph3.add_col_add_wprem(poldata_df)
poldata_df = mpf_trad_func_ph3.upd_col_waived_prem(poldata_df)
poldata_df = mpf_trad_func_ph3.upd_col_ri_ced(poldata_df, ri_ced_trad_tbl)
poldata_df = mpf_trad_func_ph3.add_col_val_type_proj(poldata_df)

poldata_df = mpf_trad_func_ph3.add_ifrs17_columns(poldata_df, benefit_code_tbl)
poldata_df = mpf_trad_func_ph3.groupby_clib_b(poldata_df, clib_list_tbl)
poldata_df_mpf = poldata_df.copy() # Assigning the dataframe to output variable
poldata_df_mpf = poldata_df_mpf.astype(mpf_trad_sch.python_output_schema())   
poldata_df_mpf.to_csv('./Outputs/Python_Phase3_output.csv')

# Add to Data Reconciliation table (after MPF transformation)
data_rec.add_row_agg(poldata_df)

2024-10-18 10:42:14: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_age_at_entry' END
2024-10-18 10:42:14: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_as_curr_mth' END
2024-10-18 10:42:14: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_as_curr_year' END
2024-10-18 10:42:14: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_mat_ben_pp' END
2024-10-18 10:42:14: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_prem_payabl_y' END
2024-10-18 10:42:14: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_bft_paybl_y' END
2024-10-18 10:42:14: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'upd_col_annual_prem' END
2024-10-18 10:42:14: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'upd_col_sum_assured_prod' END
2024-10-18 10:42:14: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_polt' END
2024-10-18 10:42:14: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_cfstat

In [13]:
# Add to Data Reconciliation table (before par cohort index)
data_rec.add_row_agg(poldata_df)

# For MPFs with PAR COHORT INDEX
poldata_df['RECORD_TO_KEEP'] = 1    # Initialize a new count of records for PAR_COHORT_INDEX MPFs
poldata_df = mpf_trad_func_ph3.add_col_par_cohort_index(poldata_df, par_table_tbl, par_channel_tbl, par_table_category_tbl)
poldata_df_par_index = poldata_df.copy()
poldata_df_par_index = poldata_df_par_index.astype(mpf_trad_sch.python_par_index_output_schema())       
poldata_df_par_index.to_csv('./Outputs/Python_Phase3_PAR_COHORT_INDEX_output.csv')

# Add to Data Reconciliation table (after par cohort index)
data_rec.add_row_agg(poldata_df)

2024-10-18 10:42:15: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_par_channel' END
2024-10-18 10:42:15: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_par_table' END
2024-10-18 10:42:15: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_retireyr_payout' END
There are policies not assigned for PAR_COHORT_INDEX.
2024-10-18 10:42:15: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_par_cohort_index' END


In [14]:
# Add to Data Reconciliation table (before asset share)
data_rec.add_row_agg(poldata_df)

# For MPFs with AS 
poldata_df = mpf_trad_func_ph3.add_col_as(poldata_df_mpf, par_table_category_tbl, asrf_tbl)
poldata_df = mpf_trad_func_ph3.upd_col_as_curr_year_mth(poldata_df, val_date_const)
poldata_df = mpf_trad_func_ph3.sort_col_as(poldata_df)

poldata_df_AS = poldata_df.copy()
poldata_df_AS = poldata_df_AS.astype(mpf_trad_sch.python_output_schema())      
poldata_df_AS.to_csv('./Outputs/Python_Phase3_AS_output.csv')

# Add to Data Reconciliation table (after asset share)
data_rec.add_row_agg(poldata_df)

There are NPAR policies assigned for PAR ETL2A.
2024-10-18 10:42:16: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'add_col_as' END
2024-10-18 10:42:16: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'upd_col_as_curr_year_mth' END
2024-10-18 10:42:16: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'sort_col_as' END


### Stage 3: Output Data

#### Output MPFs
* Outputs individual ".pro" Prophet MPF.
* Corresponds to Alteryx container 11.

In [15]:
# Read output switch and filter output data
# poldata_df_mpf = comm_func.output_filter_product(prod_output_switch, poldata_df_mpf, LOB)

# Output TRAD MPF Listing file
poldata_df_mpf.to_csv('./Outputs/MPFILES/TRAD MPF Listing.csv')         #CC: need to build additional function to drop _x0 columns; to check where LIFE_IND is included/ dropped in Alteryx
poldata_df_par_index.to_csv('./Outputs/MPFILES/TRAD MPF Listing (After PAR_COHORT_INDEX).csv')
poldata_df_AS.to_csv('./Outputs/MPFILES/TRAD MPF Listing (After PAR_ETL2A).csv')

# Output Summary Val ATRx file
summary_df = output_mpf.output_summary(poldata_df_mpf)
summary_df.to_csv('./Outputs/MPFILES/1. Before PAR_COHORT_INDEX/IFNB/Summary_Val_ATRx.csv')

summary_par_index = output_mpf.output_summary_par_index(poldata_df_par_index)
summary_par_index.to_csv('./Outputs/MPFILES/2. After PAR_COHORT_INDEX/IFNB/Summary_Val_ATRx_PAR_COHORT_INDEX.csv')

summary_as = output_mpf.output_summary_etl2a(poldata_df_AS)
summary_as.to_csv('./Outputs/MPFILES/3. After ETL2A/IFNB/Summary_Val_ATRx_ETL2A.csv')

# Data Reconciliation table after asset share
data_rec.add_row_agg(poldata_df_AS)

data_rec.output_val_table('Reconciliation_bf_par_cohort_index.csv')
data_rec.output_val_table('Reconciliation_bf_AS.csv')
data_rec.output_val_table('Reconciliation_af_AS.csv')

# MPF Generation
output_mpf.output_to_file(poldata_df_mpf, './Outputs/MPFILES/1. Before PAR_COHORT_INDEX/', 'Reconciliation_bf_par_cohort_index.csv', 'PAR normal')
output_mpf.output_to_file(poldata_df_par_index, './Outputs/MPFILES/2. After PAR_COHORT_INDEX/', 'Reconciliation_bf_AS.csv', 'PAR_COHORT_INDEX')
output_mpf.output_to_file(poldata_df_AS, './Outputs/MPFILES/3. After ETL2A/', 'Reconciliation_af_AS.csv', 'PAR_AS')

2024-10-18 10:42:16: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'output_summary' END
2024-10-18 10:42:17: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'output_summary_par_index' END
2024-10-18 10:42:17: Time elapsed: 0 Hours 00 Minutes 00 Seconds. Function 'output_summary_etl2a' END
2024-10-18 10:43:17: Time elapsed: 0 Hours 01 Minutes 00 Seconds. Function 'output_to_file' END
2024-10-18 10:43:27: Time elapsed: 0 Hours 00 Minutes 09 Seconds. Function 'output_to_file' END
2024-10-18 10:43:39: Time elapsed: 0 Hours 00 Minutes 11 Seconds. Function 'output_to_file' END


#### Output E2E Tables
* Outputs E2E tables as csv format.
* Corresponds to Alteryx container 9 (i.e. existing controls).

In [16]:
# Output BASIC_POLFEE E2E Table
basic_polfee_df.to_csv('./Outputs/E2E Table/BASIC_POLFEE.PRO', index = False)

# Output BLANKSA E2E Table
filtered_elsp_df.to_csv('./Outputs/E2E Table/ELSP_ROP_BLANKSA.PRO', index = False)
filtered_elrp_df.to_csv('./Outputs/E2E Table/ELRP_ROP_BLANKSA.PRO', index = False)
filtered_mrta_rop_df.to_csv('./Outputs/E2E Table/MRTA_ROP_BLANKSA.PRO', index = False)
filtered_mrta_df.to_csv('./Outputs/E2E Table/MRTA_BLANKSA.PRO', index = False)

# Output TOTAP_TAB E2E Table
totap_df.to_csv('./Outputs/E2E Table/TOTAP_TAB.FAC', index = False)

# Output BLANKSA_TAB E2E Table
blanksa_tab_df.to_csv('./Outputs/E2E Table/BLANKSA_TAB.FAC', index = False)

# Output PRUTERM_TAB_STAGE0 E2E Table
pruterm_tab_df_stage0.to_csv('./Outputs/E2E Table/PRUTERM_TAB_STAGE0.FAC', index = False)

# Output PRUTERM_TAB E2E Table
pruterm_tab_df.to_csv('./Outputs/E2E Table/PRUTERM_TAB.FAC', index = False)

# Output CA_Summary E2E Table
ca_summary_df.to_csv('./Outputs/E2E Table/CA_Summary.PRO', index = False)